In [105]:
import numpy as np
import pandas as pd

In [106]:
# load and shift data so score corrisponds to next day
data_combined = pd.read_csv('./data/Combined_News_DJIA.csv')
data_combined.Label = data_combined.Label.shift(1)
data_combined = data_combined[1:]
stories = [c for c in data_combined if c.startswith('Top')]
data_combined = pd.melt(data_combined, id_vars=['Date','Label'], value_vars=stories, value_name='Headline')
data_combined = data_combined.drop(['variable'], axis=1)
data_combined.head()

,Date,Label,Headline
0,2008-08-11,0.0,b'Why wont America and Nato help us? If they w...
1,2008-08-12,1.0,b'Remember that adorable 9-year-old who sang a...
2,2008-08-13,0.0,b' U.S. refuses Israel weapons to attack Iran:...
3,2008-08-14,0.0,b'All the experts admit that we should legalis...
4,2008-08-15,1.0,"b""Mom of missing gay man: Too bad he's not a 2..."


In [107]:
TT_SPLIT = .8
index = int(TT_SPLIT*len(data_combined))
train = data_combined[:index]
test = data_combined[index:]

In [108]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [109]:
trainheadlines[100]

"b'Australia refuses Bush administration request to house Guantanamo detainees'"

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [111]:
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.001, max_df=0.3, max_features = 200000, ngram_range = (3, 3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtrain.shape)

(39760, 72)


In [112]:
print(advancedvectorizer)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.3, max_features=200000, min_df=0.001,
        ngram_range=(3, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


In [113]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

X_train shape: (39760, 72)
X_test shape: (9940, 72)


In [114]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2

In [84]:
from keras.preprocessing.text import Tokenizer

# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [85]:
from keras.preprocessing import sequence
from keras.utils import np_utils

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (1590, 200)
X_test shape: (398, 200)


In [115]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import ConvLSTM2D
from keras.layers.core import Dense, Activation


print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2)) 
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, epochs=15,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
#preds15 = model.predict_classes(X_test, verbose=0)
#acc15 = accuracy_score(test['Label'], preds15)

Build model...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


Train...


ValueError: Input arrays should have the same number of samples as target arrays. Found 39760 input samples and 1590 target samples.